In [11]:
import pandas as pd
import numpy as np
import os
import xarray as xr
import rasterio
from glob import glob
import geopandas as gpd

path_fire = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\algeria_tunisia.csv"
df_fire = pd.read_csv(path_fire)

df_fire_clean = df_fire[['longitude', 'latitude', 'type']].copy()

print(df_fire_clean.head())

   longitude  latitude  type
0    9.87200  27.72818     2
1    9.77909  28.63288     2
2    9.77953  28.64012     2
3    6.25409  36.74886     2
4    4.44782  35.87978     2


## Chargement et agrégation des données climatiques par saisons + Conversion en DataFrame

In [12]:
base_dir = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\Climat\nv"
variables = ["precip", "tmin", "tmax"]

seasons_indices = {
    's1': [11, 0, 1],   # Hiver : DJF
    's2': [2, 3, 4],    # Printemps : MAM
    's3': [5, 6, 7],    # Été : JJA
    's4': [8, 9, 10]    # Automne : SON
}

all_dfs = []

for var in variables:
    print(f"\n Traitement de la variable : {var}")
    
    # Charger tous les TIF de cette variable
    input_dir = os.path.join(base_dir, f"{var}_alg_tun3")
    tif_files = sorted(glob(os.path.join(input_dir, "*.tif")))
    
    if len(tif_files) == 0:
        print(f" Aucun fichier trouvé dans {input_dir}")
        continue
    
    print(f"   → {len(tif_files)} fichiers TIF trouvés")
    
    # Ouvrir tous les TIF avec xarray (concaténés dans le temps)
    try:
        da_var = xr.open_mfdataset(tif_files, combine='nested', concat_dim='time', engine='rasterio')
        
        # Extraire le DataArray (première variable du Dataset)
        key_da = list(da_var.keys())[0]
        da_var = da_var[key_da]
        
        # Prendre seulement les 12 premiers mois (1 année complète)
        if len(tif_files) >= 12:
            da_var = da_var.isel(time=slice(0, 12))
        
        print(f"   → DataArray créé avec {da_var.sizes['time']} pas de temps")
        
        # Agréger par saison et convertir en DataFrame
        for season, months_idx in seasons_indices.items():
            # Sélectionner les mois de cette saison
            da_season = da_var.isel(time=months_idx)
            
            # Calculer l'agrégation (somme pour précip, moyenne pour températures)
            if var == "precip":
                da_aggregated = da_season.sum(dim='time')
            else:
                da_aggregated = da_season.mean(dim='time')
            
            # Convertir en DataFrame avec longitude, latitude et valeur
            df_season = da_aggregated.to_dataframe(name=f'{var}_{season}').reset_index()
            
            # Renommer les colonnes x, y en longitude, latitude
            df_season = df_season.rename(columns={'x': 'longitude', 'y': 'latitude'})
            
            # Supprimer les colonnes inutiles (band, spatial_ref, etc.)
            cols_to_keep = ['longitude', 'latitude', f'{var}_{season}']
            df_season = df_season[[col for col in cols_to_keep if col in df_season.columns]]
            
            # Supprimer les valeurs NaN et nodata
            df_season = df_season.dropna()
            df_season = df_season[df_season[f'{var}_{season}'] > -9999]
            
            all_dfs.append(df_season)
            print(f"   → {var}_{season} : converti en DataFrame ({len(df_season)} pixels valides)")
    
    except Exception as e:
        print(f"    Erreur lors du chargement de {var} : {e}")



 Traitement de la variable : precip
   → 12 fichiers TIF trouvés
   → DataArray créé avec 12 pas de temps
   → DataArray créé avec 12 pas de temps
   → precip_s1 : converti en DataFrame (54560 pixels valides)
   → precip_s1 : converti en DataFrame (54560 pixels valides)
   → precip_s2 : converti en DataFrame (54560 pixels valides)
   → precip_s2 : converti en DataFrame (54560 pixels valides)
   → precip_s3 : converti en DataFrame (54560 pixels valides)
   → precip_s3 : converti en DataFrame (54560 pixels valides)
   → precip_s4 : converti en DataFrame (54560 pixels valides)

 Traitement de la variable : tmin
   → 12 fichiers TIF trouvés
   → precip_s4 : converti en DataFrame (54560 pixels valides)

 Traitement de la variable : tmin
   → 12 fichiers TIF trouvés
   → DataArray créé avec 12 pas de temps
   → DataArray créé avec 12 pas de temps
   → tmin_s1 : converti en DataFrame (32927 pixels valides)
   → tmin_s1 : converti en DataFrame (32927 pixels valides)
   → tmin_s2 : converti en

## Fusion des DataFrames climatiques en un seul DataFrame

In [13]:
# Commencer avec le premier DataFrame
df_climat_merged = all_dfs[0]

# Fusionner tous les autres DataFrames sur longitude et latitude
for i, df in enumerate(all_dfs[1:], 1):
    df_climat_merged = pd.merge(df_climat_merged, df, on=['longitude', 'latitude'], how='inner')
    print(f"   → Fusion {i}/{len(all_dfs)-1} terminée")

print(f"\n DataFrame climatique fusionné créé : {df_climat_merged.shape}")
print(f" Colonnes : {list(df_climat_merged.columns)}")
print("\n Aperçu :")
print(df_climat_merged.head())

   → Fusion 1/11 terminée
   → Fusion 2/11 terminée
   → Fusion 3/11 terminée
   → Fusion 4/11 terminée
   → Fusion 5/11 terminée
   → Fusion 6/11 terminée
   → Fusion 5/11 terminée
   → Fusion 6/11 terminée
   → Fusion 7/11 terminée
   → Fusion 8/11 terminée
   → Fusion 9/11 terminée
   → Fusion 10/11 terminée
   → Fusion 11/11 terminée

 DataFrame climatique fusionné créé : (32927, 14)
 Colonnes : ['longitude', 'latitude', 'precip_s1', 'precip_s2', 'precip_s3', 'precip_s4', 'tmin_s1', 'tmin_s2', 'tmin_s3', 'tmin_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4']

 Aperçu :
   longitude   latitude   precip_s1  precip_s2  precip_s3   precip_s4  \
0   9.541667  37.291667  241.925003  78.650002  12.275000  129.899994   
1   9.625000  37.291667  233.375000  76.150009  12.400001  129.725006   
2   9.708333  37.291667  224.449997  73.924995  12.850000  131.099991   
3   9.791667  37.291667  216.899994  73.925003  13.075001  131.675003   
4   9.208333  37.208333  264.900024  84.475006  12.2750

## Merge final : Fire + Climat (approche optimisée avec pd.merge)

In [14]:
# Arrondir les coordonnées à la résolution de la grille climatique
# La grille climatique a un pas de 0.083333° (= 1/12°)
# On arrondit donc à 1 décimale après avoir divisé par 0.083333

resolution = 1/12  # 0.083333°

# Fonction d'arrondissement à la grille la plus proche
def round_to_grid(value, resolution):
    return np.round(value / resolution) * resolution

# Créer des copies avec coordonnées arrondies
df_fire_rounded = df_fire_clean.copy()
df_fire_rounded['longitude'] = round_to_grid(df_fire_clean['longitude'], resolution)
df_fire_rounded['latitude'] = round_to_grid(df_fire_clean['latitude'], resolution)

df_climat_rounded = df_climat_merged.copy()
df_climat_rounded['longitude'] = round_to_grid(df_climat_merged['longitude'], resolution)
df_climat_rounded['latitude'] = round_to_grid(df_climat_merged['latitude'], resolution)


print(f"   Avant : lon={df_fire_clean['longitude'].iloc[0]:.6f}, lat={df_fire_clean['latitude'].iloc[0]:.6f}")
print(f"   Après : lon={df_fire_rounded['longitude'].iloc[0]:.6f}, lat={df_fire_rounded['latitude'].iloc[0]:.6f}")

# Merge avec les coordonnées arrondies
df_final = pd.merge(
    df_fire_clean,  # Garder les coordonnées originales du feu
    df_climat_rounded,
    left_on=[df_fire_rounded['longitude'], df_fire_rounded['latitude']],
    right_on=['longitude', 'latitude'],
    how='inner'
)

# Nettoyer les colonnes dupliquées
df_final = df_final.drop(columns=['longitude', 'latitude'])
df_final = df_final.rename(columns={'key_0': 'longitude', 'key_1': 'latitude'})

print(f" Dataset final : {df_final.shape}")
print(f" Colonnes : {list(df_final.columns)}")
print("\n Aperçu :")
print(df_final.head())

   Avant : lon=9.872000, lat=27.728180
   Après : lon=9.833333, lat=27.750000
 Dataset final : (90281, 17)
 Colonnes : ['longitude_x', 'latitude_x', 'type', 'longitude_y', 'latitude_y', 'precip_s1', 'precip_s2', 'precip_s3', 'precip_s4', 'tmin_s1', 'tmin_s2', 'tmin_s3', 'tmin_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4']

 Aperçu :
   longitude_x  latitude_x  type  longitude_y  latitude_y   precip_s1  \
0      9.87200    27.72818     2     9.833333   27.750000    6.700000   
1      9.77909    28.63288     2     9.750000   28.666667    8.250000   
2      9.77953    28.64012     2     9.750000   28.666667    8.250000   
3      4.44782    35.87978     2     4.416667   35.916667  105.475006   
4      5.53337    35.70751     0     5.500000   35.666667  115.025002   

   precip_s2  precip_s3   precip_s4  tmin_s1    tmin_s2    tmin_s3    tmin_s4  \
0   8.375000   1.000000    3.200000     5.25  16.416666  26.000000  17.666666   
1  13.500000   1.700000    4.500000     4.00  14.666667  24.33